

# Bunraku Full (On-/Off-line) Collection 
__Data Transformation SQL CSVs -> JSON__

<hr/>

### Data files to generate:

- __authors: 123__
- __characters: 2,107__ — 17,006 images
- __images: 21268__
- __kashira: 129__ — 17,019 images
- __performances: 931__ — 18,533 images
- __performers: 184__ — 14,893 images
- __plays: 178__ — 18,791 images
- __productions: 293__ — 18,750 images
- __pscenes: 2,609__ — 18,000 images
- __spucks: 16,625__
- __tags: 76__ — 8,941 images



# Setup
<hr/>

In [1]:
import pandas as pd
pd.set_option('max_colwidth',1000)
pd.set_option('max_seq_items','none')
pd.set_option('display.max_colwidth',50)

In [2]:
# import data as dtype=object to avoid NaN/Float conversion of IDs
authors = pd.read_csv('in/authors.csv', dtype=object)
characters = pd.read_csv('in/characters.csv', dtype=object)
images = pd.read_csv('in/allimages.csv', dtype=object)
kashira = pd.read_csv('in/kashira.csv', dtype=object)
performances = pd.read_csv('in/performances.csv', dtype=object)
performers = pd.read_csv('in/performers.csv', dtype=object)
plays = pd.read_csv('in/plays.csv', dtype=object)
productions = pd.read_csv('in/productions.csv', dtype=object)
pscenes = pd.read_csv('in/scenes_productions.csv', dtype=object)
scenes = pd.read_csv('in/scenes.csv', dtype=object)
shamisenplayers = pd.read_csv('in/sceneshamisens.csv', dtype=object)
musicians = pd.read_csv('in/scenekotokokyus.csv', dtype=object)
narrators = pd.read_csv('in/scenetayus.csv', dtype=object)
spucks = pd.read_csv('in/spucks.csv', dtype=object)
tags = pd.read_csv('in/tags.csv', dtype=object)
# import + drop duplicates on join tables
authors_plays = pd.read_csv('in/authors_plays.csv', dtype=object).drop_duplicates()
characters_images = pd.read_csv('in/characters_images.csv', dtype=object).drop_duplicates()
characters_plays = pd.read_csv('in/characters_plays.csv', dtype=object).drop_duplicates()
kashira_images = pd.read_csv('in/kashira_images.csv', dtype=object).drop_duplicates()
kashira_plays = pd.read_csv('in/kashira_plays.csv', dtype=object).drop_duplicates()
performances_images = pd.read_csv('in/performances_images.csv', dtype=object).drop_duplicates()
performers_images = pd.read_csv('in/performers_images.csv', dtype=object).drop_duplicates()
plays_images = pd.read_csv('in/plays_images.csv', dtype=object).drop_duplicates()
productions_images = pd.read_csv('in/productions_images.csv', dtype=object).drop_duplicates()
scenes_images = pd.read_csv('in/scenes_images.csv', dtype=object).drop_duplicates()
tags_images = pd.read_csv('in/tags_images.csv', dtype=object).drop_duplicates()

### Authors

In [3]:
authors = authors.merge(authors_plays.groupby('author_id')['play_id'].apply(list).reset_index(), how='left')

### Characters

In [4]:
# add  plays
characters = characters.merge(characters_plays.groupby('character_id')['play_id'].apply(list).reset_index(), how='left')
# add images
characters = characters.merge(characters_images.groupby('character_id')['image_id'].apply(list).reset_index(), how='left')
characters = characters.drop('character_code',1)

### Images

In [5]:
# add characters
images = images.merge(characters_images.groupby('image_id')['character_id'].apply(list).reset_index(), how='left')
# add kashira
images = images.merge(kashira_images.groupby('image_id')['kashira_id'].apply(list).reset_index(), how='left')
# add performances
images = images.merge(performances_images.groupby('image_id')['performance_id'].apply(list).reset_index(), how='left')
# add performers
images = images.merge(performers_images.groupby('image_id')['performer_id'].apply(list).reset_index(), how='left')
# add plays
images = images.merge(plays_images.groupby('image_id')['play_id'].apply(list).reset_index(), how='left')
# add productions
images = images.merge(productions_images.groupby('image_id')['production_id'].apply(list).reset_index(), how='left')
# add scenes
images = images.merge(scenes_images.groupby('image_id')['pscene_id'].apply(list).reset_index(), how='left')
# add tags
images = images.merge(tags_images.groupby('image_id')['tag_id'].apply(list).reset_index(), how='left')

images = images[['image_id','media_type','character_id','tag_id','kashira_id','performance_id','performer_id','play_id','production_id','pscene_id','container','container_type','creator','item_id','colser_id','notes','objid','sequence','series','slidepage_folder']]


### Kashira (puppets)

In [6]:
# add images
kashira = kashira.merge(kashira_images.groupby('kashira_id')['image_id'].apply(list).reset_index(), how='left')
# add plays
kashira = kashira.merge(kashira_plays.groupby('kashira_id')['play_id'].apply(list).reset_index(), how='left')

kashira = kashira[['kashira_id','label_eng','label_ka','category','image_id','play_id','sort_ja']]


### (P)Scenes (at performance level)

In [7]:
# add labels and scene_order
scenes = scenes[['scene_id','label_eng','label_ja','label_ka','scene_order']]
pscenes = pscenes.drop('spuck_note',1).drop('tayu_shamisen_note',1)
pscenes = pscenes.merge(scenes, how='left')
# add spuck_id
pscenes = pscenes.merge(spucks.groupby('pscene_id')['spuck_id'].apply(list).reset_index(), how='left')
# drop narrator with id 0 (no such performer exists)
narrators = narrators[narrators['narrator_id'] != '0']
# add narrator_ids
pscenes = pscenes.merge(narrators.groupby('pscene_id')['narrator_id'].apply(list).reset_index(), how='left')
# add musician_ids
pscenes = pscenes.merge(musicians.groupby('pscene_id')['musician_id'].apply(list).reset_index(), how='left')
# add shamisen_ids
pscenes = pscenes.merge(shamisenplayers.groupby('pscene_id')['shamisen_id'].apply(list).reset_index(), how='left')
# add image_ids
pscenes = pscenes.merge(scenes_images.groupby('pscene_id')['image_id'].apply(list).reset_index(), how='left')

### Performances

In [8]:
performances =  performances.drop('data_id',1).drop('code',1)
# add images
performances = performances.merge(performances_images.groupby('performance_id')['image_id'].apply(list).reset_index(), how='left')
# add pscenes
performances = performances.merge(pscenes.groupby('performance_id')['pscene_id'].apply(list).reset_index(), how='left')
# add characters from play_id
performances = performances.merge(characters_plays.groupby('play_id')['character_id'].apply(list).reset_index(), how='left')

### Plays

In [9]:
# authors
plays = plays.merge(authors_plays.groupby('play_id')['author_id'].apply(list).reset_index(), how='left')
# characters
plays = plays.merge(characters_plays.groupby('play_id')['character_id'].apply(list).reset_index(), how='left')
# images
plays = plays.merge(plays_images.groupby('play_id')['image_id'].apply(list).reset_index(), how='left')
# productions
plays = plays.merge(performances.groupby('play_id')['production_id'].apply(list).reset_index(), how='left')
# performances
plays = plays.merge(performances.groupby('play_id')['performance_id'].apply(list).reset_index(), how='left')

### Productions

In [10]:
productions = productions.drop('performance_num',1)
# images
productions = productions.merge(productions_images.groupby('production_id')['image_id'].apply(list).reset_index(), how='left')
# performances
productions = productions.merge(performances.groupby('production_id')['performance_id'].apply(list).reset_index(), how='left')
# plays
productions = productions.merge(performances.groupby('production_id')['play_id'].apply(list).reset_index(), how='left')
productions = productions[['production_id','dates','place','label_eng','image_id','performance_id','play_id']]

### Tags

In [12]:
tags['notes'] = tags['notes'].replace({r'\n': ''}, regex=True)
tags = tags.merge(tags_images.groupby('tag_id')['image_id'].apply(list).reset_index(), how='left')

### Performers

In [13]:
performers = performers[['performer_id','name_proper','alt_name','name_ka','alt_name_ka','specialty','dates','notes']]
# add images
performers = performers.merge(performers_images.groupby('performer_id')['image_id'].apply(list).reset_index(), how='left')
# add performances as musician 
### make a join table from pscenes
performer_as_musician = pscenes[['musician_id','performance_id']].dropna(how='any').rename(columns={'musician_id':'performer_id','performance_id':'musician_perf_id'})
performer_as_musician = performer_as_musician.groupby('musician_perf_id').performer_id.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)
performer_as_musician.columns = ['musician_perf_id','performer_id']
performer_as_musician.drop_duplicates(inplace=True)
### add performances
performers = performers.merge(performer_as_musician.groupby('performer_id')['musician_perf_id'].apply(list).reset_index(), how='left')
# add performances as narrator
### make a join table from pscenes
performer_as_narrator = pscenes[['narrator_id','performance_id']].dropna(how='any').rename(columns={'narrator_id':'performer_id','performance_id':'narrator_perf_id'})
performer_as_narrator = performer_as_narrator.groupby('narrator_perf_id').performer_id.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)
performer_as_narrator.columns = ['narrator_perf_id','performer_id']
performer_as_narrator.drop_duplicates(inplace=True)
### add performances
performers = performers.merge(performer_as_narrator.groupby('performer_id')['narrator_perf_id'].apply(list).reset_index(), how='left')
# add performances as shamisen player
### make a join table from pscenes
performer_as_shamisen = pscenes[['shamisen_id','performance_id']].dropna(how='any').rename(columns={'shamisen_id':'performer_id','performance_id':'shamisen_perf_id'})
performer_as_shamisen = performer_as_shamisen.groupby('shamisen_perf_id').performer_id.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)
performer_as_shamisen.columns = ['shamisen_perf_id','performer_id']
performer_as_shamisen.drop_duplicates(inplace=True)
### add performances
performers = performers.merge(performer_as_shamisen.groupby('performer_id')['shamisen_perf_id'].apply(list).reset_index(), how='left')
# add performances as puppeteer and kashira used
### make a join table from pscenes
spucks_performances = pscenes[['spuck_id','performance_id']].dropna(how='any').rename(columns={'performance_id':'puppeteer_perf_id'})
spucks_performances = spucks_performances.groupby('puppeteer_perf_id').spuck_id.apply(lambda x: pd.DataFrame(x.values[0])).reset_index().drop('level_1', axis = 1)
spucks_performances.columns = ['puppeteer_perf_id','spuck_id']
spucks_performances.drop_duplicates(inplace=True)
### make join table with performer_id, kashira_id, and puppeteer_perf_id
xtra_spucks = spucks[['spuck_id','pscene_id','puppeteer_id','kashira_id']].rename(columns={'puppeteer_id':'performer_id'})
xtra_spucks = xtra_spucks.merge(spucks_performances, on='spuck_id', how='left').drop('spuck_id',1).drop('pscene_id',1)
### add performances
performer_as_puppeteer = xtra_spucks[['performer_id','puppeteer_perf_id']].dropna(how='any').drop_duplicates()
performers = performers.merge(performer_as_puppeteer.groupby('performer_id')['puppeteer_perf_id'].apply(list).reset_index(), how='left')
### add kashira
performer_puppets = xtra_spucks[['performer_id','kashira_id']].dropna(how='any').drop_duplicates()
performers = performers.merge(performer_puppets.groupby('performer_id')['kashira_id'].apply(list).reset_index(), how='left')

# Export to CSV and JSON

In [ ]:
# replace descriptive id (used for joins) with generic id
authors.rename(columns={'author_id':'id'}, inplace=True)
characters.rename(columns={'character_id':'id'}, inplace=True)
images.rename(columns={'image_id':'id'}, inplace=True)
kashira.rename(columns={'kashira_id':'id'}, inplace=True)
performances.rename(columns={'performance_id':'id'}, inplace=True)
performers.rename(columns={'performer_id':'id'}, inplace=True)
plays.rename(columns={'play_id':'id'}, inplace=True)
productions.rename(columns={'production_id':'id'}, inplace=True)
pscenes.rename(columns={'pscene_id':'id'}, inplace=True)
spucks.rename(columns={'spuck_id':'id'}, inplace=True)
tags.rename(columns={'tag_id':'id'}, inplace=True)

In [ ]:
# export data as CSV
authors.to_csv('authors_full.csv', encoding='utf8', index=False)
characters.to_csv('characters_full.csv', encoding='utf8', index=False)
images.to_csv('images_full.csv', encoding='utf8', index=False)
kashira.to_csv('kashira_full.csv', encoding='utf8', index=False)
performances.to_csv('performances_full.csv', encoding='utf8', index=False)
performers.to_csv('performers_full.csv', encoding='utf8', index=False)
plays.to_csv('plays_full.csv', encoding='utf8', index=False)
productions.to_csv('productions_full.csv', encoding='utf8', index=False)
pscenes.to_csv('pscenes_full.csv', encoding='utf8', index=False)
spucks.to_csv('spucks_full.csv', encoding='utf8', index=False)
tags.to_csv('tags_full.csv', encoding='utf8', index=False)

In [ ]:
# export data as JSON
authors.to_json('authors_full.json', orient="records", force_ascii=False)
characters.to_json('characters_full.json', orient="records", force_ascii=False)
creators.to_json('creators_full.json', orient="records", force_ascii=False)
images.to_json('images_full.json', orient="records", force_ascii=False)
kashira.to_json('kashira_full.json', orient="records", force_ascii=False)
performances.to_json('performances_full.json', orient="records", force_ascii=False)
performers.to_json('performers_full.json', orient="records", force_ascii=False)
plays.to_json('plays_full.json', orient="records", force_ascii=False)
productions.to_json('productions_full.json', orient="records", force_ascii=False)
pscenes.to_json('pscenes_full.json', orient="records", force_ascii=False)
spucks.to_json('spucks_full.json', orient="records", force_ascii=False)
tags.to_json('tags_full.json', orient="records", force_ascii=False)